In [29]:
#packages
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import matplotlib.pyplot as plt
import themepy
import matplotlib.font_manager
from IPython.core.display import HTML
from highlight_text import HighlightText, ax_text, fig_text
import matplotlib.lines as lines

In [13]:
#fonts
def make_html(fontname):
    return "<p>{font}: <span style='font-family:{font}; font-size: 24px;'>{font}</p>".format(font=fontname)

code = "\n".join([make_html(font) for font in sorted(set([f.name for f in matplotlib.font_manager.fontManager.ttflist]))])

In [14]:
#load file from initial folder
df=pd.read_csv("...")

In [15]:
#setting theme previously made
theme = themepy.Theme()
theme.set_theme("pavel_dark")

pavel_dark is the active theme

In [48]:
#function for radar plot + stats
def make_radar(df,labels, player1, player2,position):

    color1="#FADA5E"
    color2="#FF69B4"
    #dataframes
    max=df["minutes_90s"].max()
    df=df[df["minutes_90s"]>(max*0.1)]
    df_position=df[df['position'].str.contains(position)]
    df_resized=df_position[labels]


    #per90 stats
    labels_stats=list(labels)
    labels_stats.remove("player")
    labels_stats.remove("minutes_90s")
    labels_stats=list(labels_stats)
    for label in labels_stats:
        df_resized[(label+"_per90")]=df_resized[label]/df_resized["minutes_90s"]
    labels_stats=np.array(labels_stats)

    #percentiles
    df_resized=df_resized.drop(labels_stats,axis=1).round(decimals=3)
    labels_per90=list(df_resized.columns)
    labels_per90.remove("player")
    labels_per90.remove("minutes_90s")
    labels_per90=list(labels_per90)
    labels_per90=np.array(labels_per90)
    df_values= df_resized.loc[:, df_resized.columns != 'player'].round(decimals=2)
    df_values=pd.concat([df_resized["player"],df_values],axis=1)
    df_values_dec3= df_resized.loc[:, df_resized.columns != 'player'].round(decimals=3)
    df_values_dec3=pd.concat([df_resized["player"],df_values_dec3],axis=1)
    df= df_resized.loc[:, df_resized.columns != 'player'].rank(pct=True).round(decimals=2)
    df=pd.concat([df_resized["player"],df],axis=1)

    #final dfs
    p1=df[df["player"]==player1]
    p2=df[df["player"]==player2]
    p1_p2=pd.concat([p1,p2],axis=0)
    df_resized["group"]="grey"
    p1["group"]="blue"
    p2["group"]="red"
    p1_p2=p1_p2.reset_index()
    p1_values=df_values[df_values["player"]==player1]
    p2_values=df_values[df_values["player"]==player2]
    p12=pd.concat([p1,p2],axis=0)
    p1_p2_values=pd.concat([p1_values,p2_values],axis=0)
    p1_p2_values.loc['median'] = df_values.median().round(decimals=2)
    cell_text = p1_p2_values[labels_per90].values.transpose().tolist()
    

    #PLOT 1 - RADAR
    N = len(labels_per90)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]

    px = 1/plt.rcParams['figure.dpi']
    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(1600*px, 900*px),
                       gridspec_kw={
                           'width_ratios': [1, 1],
                       'wspace': 0})

    plt.subplots_adjust(left=0.3, bottom=0.2, right=0.7, top=0.8)

 
    ax = plt.subplot(122, polar=True)
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction(-1)
 
    ax.set_xticks(angles[:-1], labels_stats, ha="center",va="center",
    color='w', size=12, fontname='Open Sans', fontweight='medium',zorder=3,alpha=0.8,position=(0.25,0.25))
    ax.set_theta_offset(np.pi / 2)
    ax.set_theta_direction('clockwise')
 
    ax.set_rlabel_position(30)
    ax.set_yticks([0.2,0.4,0.6,0.8,1.0], ['20th','40th','60th','80th','100th'], color="w", alpha=0.6,
               size=10, fontname='Open Sans', fontweight='medium',zorder=2 )
    ax.set_ylim(0,1.4)
    ax.set_position([0.5, 0.2, 0.35, 0.6])
 
    # First player
    values=p1_p2.loc[0,labels_per90].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, linestyle='solid', label="Player 1", color=color1)
    ax.fill(angles, values, color1, alpha=0.25)
    
    # Second player
    values=p1_p2.loc[1,labels_per90].values.flatten().tolist()
    values += values[:1]
    ax.plot(angles, values, linewidth=2, linestyle='solid', label="Player 2", color=color2)
    ax.fill(angles, values, color2, alpha=0.25)
    
    ax.grid(color='grey',zorder=1)
    ax.spines['polar'].set_color(theme.background)
    ax.spines['polar'].set_alpha(0)
    ax.set_facecolor(theme.background)
    

    transform = ax.transAxes
    rec = plt.Rectangle((0, 0), (1), (1), fill=False, lw=1,transform=transform,color="w",zorder=4)
    rec = ax.add_patch(rec)
    rec.set_clip_on(False)



    #2 PLOT - TABLE
    ax2 = plt.subplot(121)
    plt.rcParams.update({'font.family':'Open Sans'})
    plt.rcParams.update({'text.color':'white'})
    ax2.axis('off')
    ax2.axis('tight')
    ax2.xaxis.set_visible(False) 
    ax2.yaxis.set_visible(False)
    the_table = plt.table(cellText=cell_text,
                        rowLoc="center",cellLoc="center",colLoc="center", bbox=[0.0, 0.0, 1, 1],
                        rowLabels=list(labels_stats), loc='center',fontsize=12,
                        rowColours=list(np.repeat(theme.background,6)),
                        cellColours=[list(np.repeat(theme.background,3))]*6)
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(12)
    for cell in the_table.get_children():
        cell_text = cell.get_text().get_text()
        cell.set_edgecolor("w")
        the_table.set_alpha(0.8)

    

    for i in range(0,6):
        the_cell = the_table[i,0]
        the_text = the_cell.get_text()
        the_text.set_color(color1)
        the_text.set_alpha(0.8)
    for i in range(0,6):
        the_cell = the_table[i,1]
        the_text = the_cell.get_text()
        the_text.set_color(color2)
        the_text.set_alpha(0.8)
    for i in range(0,6):
        the_cell = the_table[i,2]
        the_text = the_cell.get_text()
        the_text.set_color("grey")
        


    fig.text(0.5, 0.05, "Info: Stats (Per 90) | Radar Chart (Percentiles)\nData: FBREF via Statsbomb\nMade By: @pawellkaroll", 
                    ha = 'center',color = "w", alpha=0.4,
                    font={'size':12,"style":"italic"})
    fig.text(0.5, 0.95, "Premier League, season 2021/2022", 
                    ha = 'center',color = "w", alpha=0.4,
                    font={'size':14,'family':"Open Sans","weight":"medium"})
    fig.text(0.5, 0.86, "Who is a better fit for Arsenal?", 
                    ha = 'center',color = "w", alpha=0.8,
                    font={'size':30,'family':"Open Sans","weight":"bold"})
    fig_text(x = 0.5, y = 0.83,s="<Eddie Nketiah>< vs ><Alexandre Lacazette>",ha='center', va='center',
        highlight_textprops=[{"color": color1,'size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                                {"color": 'white','size':20,'family':"Open Sans","weight":"bold","alpha":0.6},
                             {"color": color2,'size':20,'family':"Open Sans","weight":"bold","alpha":0.6}])


    file="..."+player1+"_"+player2+".png"
    plt.savefig(file)  

In [50]:
labels=["player","minutes_90s","npxg","xa","sca","progressive_carries","passes_received","pressures"]
#make_radar(df=df,labels=labels,player1="Eddie Nketiah",player2="Alexandre Lacazette",position="FW")